In [ ]:
import pandas as pd
import numpy as np
from itertools import chain

# Hi-C utilities imports:
import cooler
import bioframe
import cooltools
from cooltools.lib.numutils import fill_diag

# Visualization imports:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm
from matplotlib import cm
# %matplotlib widget
%matplotlib inline

import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cooltools.lib import plotting

from itertools import cycle

# # from ipywidgets import interact, fixed
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets

# enable editable text ...
mpl.rcParams["pdf.fonttype"]=42
mpl.rcParams["svg.fonttype"]="none"
mpl.rcParams['axes.linewidth'] = 0.5

### Import modified "guts" of the dotfinder submodule from `helper_func` file

In [ ]:
from helper_func import draw_kernel

# turns out still need some of the dotfinder guts in here
from cooltools.api.dotfinder import bp_to_bins, generate_tiles_diag_band
from cooltools.lib.numutils import LazyToeplitz
from cooltools.lib.common import assign_regions

from functools import partial
from data_catalog import bws, bws_vlim, telo_dict

import datashader as ds, xarray as xr
from datashader import transfer_functions as tf, reductions as rd
from datashader.mpl_ext import dsshow, alpha_colormap

from matplotlib.cm import YlOrRd
import matplotlib.image as mpimg

from palettable.scientific import sequential
lajolla_r = sequential.LaJolla_10_r.get_mpl_colormap()
bilbao_r = sequential.Bilbao_20_r.get_mpl_colormap()
batlow_r = sequential.Batlow_20_r.get_mpl_colormap()
devon_r = sequential.Devon_20_r.get_mpl_colormap()
davos_r = sequential.Davos_20_r.get_mpl_colormap()
# fall_cmap = fall
lajolla = sequential.LaJolla_10.get_mpl_colormap()
bilbao = sequential.Bilbao_20.get_mpl_colormap()
batlow = sequential.Batlow_20.get_mpl_colormap()
devon = sequential.Devon_20.get_mpl_colormap()
davos = sequential.Davos_20.get_mpl_colormap()

imola_r = sequential.Imola_20_r.get_mpl_colormap()
imola = sequential.Imola_20.get_mpl_colormap()

nuuk_r = sequential.Nuuk_20_r.get_mpl_colormap()
nuuk = sequential.Nuuk_20.get_mpl_colormap()

hawaii_r = sequential.Hawaii_20_r.get_mpl_colormap()
hawaii = sequential.Hawaii_20.get_mpl_colormap()

oleron_r = sequential.Oleron_20_r.get_mpl_colormap()
oleron = sequential.Oleron_20.get_mpl_colormap()

oslo_r = sequential.Oslo_20_r.get_mpl_colormap()
oslo = sequential.Oslo_20.get_mpl_colormap()

bamako_r = sequential.Bamako_20_r.get_mpl_colormap()
bamako = sequential.Bamako_20.get_mpl_colormap()

grayc_r = sequential.GrayC_20_r.get_mpl_colormap()
grayc = sequential.GrayC_20.get_mpl_colormap()

lapaz_r = sequential.LaPaz_20_r.get_mpl_colormap()
lapaz = sequential.LaPaz_20.get_mpl_colormap()


In [ ]:
# import matplotlib.pyplot as plt
import bbi
import matplotlib.lines as lines
from matplotlib.lines import Line2D
from matplotlib.patches import ConnectionPatch, Rectangle
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.inset_locator import BboxConnector

# manual colorbar example ...
# #
# cax = divider.append_axes("right", size=0.1, pad=0.05)
# plt.colorbar(
#     cm.ScalarMappable(norm=PowerNorm(1/3., vmin=vmin, vmax=vmax), cmap=mycmap),
#     cax=cax,
# )

import matplotlib.patches as mpatches
import matplotlib.path as mpath

# import matplotlib.pyplot as plt
import bbi
import matplotlib.lines as lines
from matplotlib.lines import Line2D
from matplotlib.patches import ConnectionPatch, Rectangle
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.inset_locator import BboxConnector


# draw 2 arm chromosome ....
def get_chrom_path(
    x0y0,  # position of the centromere ...
    parm,  # p arm length
    qarm,  # q arm length
    width,  # width
    color="grey",
):
    # unpack coordinates of the center (centromere)
    x0, y0 = x0y0

    # calculate half width
    hw = 0.5*width

    dtelo = 0.5*width
    dxcent = 0.2*width
    dycent = 0.5*width

    # Prepare the data for the PathPatch below.
    Path = mpath.Path
    codes, verts = zip(*[
        # p-arm up:
        (Path.MOVETO, [x0-hw, y0+dycent]),
        (Path.LINETO, [x0-hw, y0+parm-dtelo]),
        # p-telomere curve:
        (Path.CURVE3, [x0-hw, y0+parm]),
        (Path.CURVE3, [x0, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
        # p-arm down:
        (Path.LINETO, [x0+hw, y0+dycent]),
        # centromere S curve (right)
        # (1S)
        (Path.CURVE4, [x0+hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0]),
        # (2S)
        (Path.CURVE4, [x0+hw-dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-dycent]),
        # q-arm down
        (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
        # q-telomere curve:
        (Path.CURVE3, [x0+hw, y0-qarm]),
        (Path.CURVE3, [x0, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
        # q-arm up
        (Path.LINETO, [x0-hw, y0-dycent]),
        # centromere S curve (left)
        (Path.CURVE4, [x0-hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0]),
        (Path.CURVE4, [x0-hw+dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+dycent]),
        (Path.CLOSEPOLY, [x0-hw, y0+dycent]),
    ])

    chrom = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
    chrom.set(color=color)
    return chrom






# draw 2 arm chromosome ....
def get_chrom_path_wpathes(
    x0y0,  # position of the centromere ...
    parm,  # p arm length
    qarm,  # q arm length
    width,  # width
    color="grey",
    patches=None,  # [(_arm, _from, _to, pcolor)]
):
    # unpack coordinates of the center (centromere)
    x0, y0 = x0y0

    # calculate half width
    hw = 0.5*width

    dtelo = 0.5*width
    dxcent = 0.2*width
    dycent = 0.5*width

    # Prepare the data for the PathPatch below.
    Path = mpath.Path
    codes, verts = zip(*[
        # p-arm up:
        (Path.MOVETO, [x0-hw, y0+dycent]),
        (Path.LINETO, [x0-hw, y0+parm-dtelo]),
        # p-telomere curve:
        (Path.CURVE3, [x0-hw, y0+parm]),
        (Path.CURVE3, [x0, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
        # p-arm down:
        (Path.LINETO, [x0+hw, y0+dycent]),
        # centromere S curve (right)
        # (1S)
        (Path.CURVE4, [x0+hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0]),
        # (2S)
        (Path.CURVE4, [x0+hw-dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-dycent]),
        # q-arm down
        (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
        # q-telomere curve:
        (Path.CURVE3, [x0+hw, y0-qarm]),
        (Path.CURVE3, [x0, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
        # q-arm up
        (Path.LINETO, [x0-hw, y0-dycent]),
        # centromere S curve (left)
        (Path.CURVE4, [x0-hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0]),
        (Path.CURVE4, [x0-hw+dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+dycent]),
        (Path.CLOSEPOLY, [x0-hw, y0+dycent]),
    ])

    chrom = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
    chrom.set(color=color)

    patches_to_draw = []
    if patches is not None:
        # go over patches ...
        for _arm, _from, _to, pcolor in patches:
            hw = hw + 0.005
            if _arm == "q":
                codes, verts = zip(*[
                    # rectangle
                    (Path.MOVETO, [x0-hw, y0-_from]),
                    (Path.LINETO, [x0-hw, y0-_to]),
                    (Path.LINETO, [x0+hw, y0-_to]),
                    (Path.LINETO, [x0+hw, y0-_from]),
                    (Path.CLOSEPOLY, [x0-hw, y0-_from]),
                ])
            elif _arm == "p":
                codes, verts = zip(*[
                    # rectangle
                    (Path.MOVETO, [x0-hw, y0+_from]),
                    (Path.LINETO, [x0-hw, y0+_to]),
                    (Path.LINETO, [x0+hw, y0+_to]),
                    (Path.LINETO, [x0+hw, y0+_from]),
                    (Path.CLOSEPOLY, [x0-hw, y0+_from]),
                ])
            else:
                raise("arms !!!")
            part = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
            part.set(color=pcolor)
            patches_to_draw.append(part)

    return chrom, patches_to_draw





# draw 2 arm chromosome ....
def get_chrom_path_highlight(
    x0y0,  # position of the centromere ...
    parm,  # p arm length
    qarm,  # q arm length
    width,  # width
    color="grey",
    pshade=0,  # fraction of p arm to be highlighted
    qshade=0,  # fraction of q arm to be highlighted
    colorp="black",
    colorq="black",
):
    """
    a function that also highlight terminal part of
    p and q arms ...
    """
    # unpack coordinates of the center (centromere)
    x0, y0 = x0y0

    # calculate half width
    hw = 0.5*width

    dtelo = 0.5*width
    dxcent = 0.2*width
    dycent = 0.5*width

    # Prepare the data for the PathPatch below.
    Path = mpath.Path
    codes, verts = zip(*[
        # p-arm up:
        (Path.MOVETO, [x0-hw, y0+dycent]),
        (Path.LINETO, [x0-hw, y0+parm-dtelo]),
        # p-telomere curve:
        (Path.CURVE3, [x0-hw, y0+parm]),
        (Path.CURVE3, [x0, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
        # p-arm down:
        (Path.LINETO, [x0+hw, y0+dycent]),
        # centromere S curve (right)
        # (1S)
        (Path.CURVE4, [x0+hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0]),
        # (2S)
        (Path.CURVE4, [x0+hw-dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-dycent]),
        # q-arm down
        (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
        # q-telomere curve:
        (Path.CURVE3, [x0+hw, y0-qarm]),
        (Path.CURVE3, [x0, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
        # q-arm up
        (Path.LINETO, [x0-hw, y0-dycent]),
        # centromere S curve (left)
        (Path.CURVE4, [x0-hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0]),
        (Path.CURVE4, [x0-hw+dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+dycent]),
        (Path.CLOSEPOLY, [x0-hw, y0+dycent]),
    ])
    chrom = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
    chrom.set(color=color)

    hw = hw + 0.005
    if pshade > 0:
        dy_pshade = (1.0-pshade)*parm
        if dy_pshade < dycent:
            dy_pshade = dycent
        # p -arm highlight
        codes, verts = zip(*[
            # p-arm up:
            (Path.MOVETO, [x0-hw, y0+dy_pshade]),
            (Path.LINETO, [x0-hw, y0+parm-dtelo]),
            # p-telomere curve:
            (Path.CURVE3, [x0-hw, y0+parm]),
            (Path.CURVE3, [x0, y0+parm]),
            (Path.CURVE3, [x0+hw, y0+parm]),
            (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
            # p-arm down:
            (Path.LINETO, [x0+hw, y0+dy_pshade]),
            (Path.CLOSEPOLY, [x0-hw, y0+dy_pshade]),
        ])
        part = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
        part.set(color=colorp)
    else:
        part = None
        dy_pshade = 0

    if qshade > 0:
        # Prepare the data for the PathPatch below.
        dy_qshade = (1.0-qshade)*qarm
        if dy_qshade < dycent:
            dy_qshade = dycent
        codes, verts = zip(*[
            (Path.MOVETO, [x0+hw, y0-dy_qshade]),
            # q-arm down
            (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
            # q-telomere curve:
            (Path.CURVE3, [x0+hw, y0-qarm]),
            (Path.CURVE3, [x0, y0-qarm]),
            (Path.CURVE3, [x0-hw, y0-qarm]),
            (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
            # q-arm up
            (Path.LINETO, [x0-hw, y0-dy_qshade]),
            (Path.CLOSEPOLY, [x0+hw, y0-dy_qshade]),
        ])
        qart = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
        qart.set(color=colorq)
    else:
        qart = None
        dy_qshade = 0

    return chrom, (part, y0+dy_pshade), (qart, y0-dy_qshade)


In [ ]:
import numpy as np
from scipy import interpolate
# https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array
def fill_nan(A):
    '''
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

### pick a dataset and binsize to work on ...

In [ ]:

telo_sampled_dict = {
    'mMito': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-0-Ms-R1R2.hg38.mapq_30.1000.sampled.mcool',
    'mTelo': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-0-Telo-R2.hg38.mapq_30.1000.sampled.mcool',
    'mCyto': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-0-Cyto-R1.hg38.mapq_30.1000.sampled.mcool',
    'm5hR1R2': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-0-G1s-R1R2.hg38.mapq_30.1000.sampled.mcool',
    'm10hR1R2': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-0-10hR-R1R3.hg38.mapq_30.1000.sampled.mcool',
    'pMito': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-2-Ms-R1R2.hg38.mapq_30.1000.sampled.mcool',
    'pTelo': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-4-Telo-R2.hg38.mapq_30.1000.sampled.mcool',
    'pCyto': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-4-Cyto-R1.hg38.mapq_30.1000.sampled.mcool',
    'p5hR1R2': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-7-G1s-R1R2.hg38.mapq_30.1000.sampled.mcool',
    'p10hR1R2': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-12-10hR-R1R3.hg38.mapq_30.1000.sampled.mcool',
    'mp10hR1R2': '/abyss/sergpolly/data_ranger/sampledcoolers/ranGAP1-7-10hR-R1R3.hg38.mapq_30.1000.sampled.mcool',
    'N93m5': '/abyss/sergpolly/data_ranger/sampledcoolers/N93-0-G1s-R1R2.hg38.mapq_30.1000.sampled.mcool',
    'N93m10': '/abyss/sergpolly/data_ranger/sampledcoolers/N93-0-10hR-G1s-R3R4.hg38.mapq_30.1000.sampled.mcool',
    'N93p5': '/abyss/sergpolly/data_ranger/sampledcoolers/N93-7-G1s-R1R2.hg38.mapq_30.1000.sampled.mcool',
    'N93p10': '/abyss/sergpolly/data_ranger/sampledcoolers/N93-12-10hR-G1s-R3R4.hg38.mapq_30.1000.sampled.mcool',
    'N93mp10': '/abyss/sergpolly/data_ranger/sampledcoolers/N93-7-10hR-G1s-R3R4.hg38.mapq_30.1000.sampled.mcool',
}


In [ ]:
# ! ls /data/proj_sync/data_ranger/finalcoolers_mega/ranGAP1-aux-G1s-MEGA.hg38.mapq_30.1000.mcool

# 10 kb is a resolution at which one can clearly see "dots":
binsize = 10_000
# cooler files that we'll work on :
telo_clrs = { _k: cooler.Cooler(f"{_path}::/resolutions/{binsize}") for _k, _path in telo_sampled_dict.items() }

### define arms for the schematic ...

In [ ]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
hg38_arms_full = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
arms_dict = hg38_arms_full.set_index("name").eval("(end - start)//1000000").to_dict()

## Figure out anchors drawing and highlighting here ...

In [ ]:
id_anchor_fnames = {
    "mega_2X_enrichment": "ID_anchors/mega_2X_enrichment.fourth_mega.max_size.bed",
    "5hr_2X_enrichment_old": "ID_anchors/5hr_2X_enrichment.second_bulk.max_size.bed",
    "5hr_2X_enrichment": "ID_anchors/5hr_2X_enrichment.pixel_derived.bed",
    "5hr_2X_enrichment_nosing": "ID_anchors/5hr_2X_enrichment.pixel_derived.no_singletons.bed",
    "5hr_2X_enrichment_signal": "ID_anchors/5hr_2X_enrichment.pixel_derived.signal_peaks.bed",
    "10hr_2X_enrichment_signal": "ID_anchors/10hrs_2X_enrichment.pixel_derived.signal_peaks.bed",
    "N93p5_2X_enrichment_signal": "ID_anchors/N93p5_2X_enrichment.pixel_derived.signal_peaks.bed",
    "pCyto_2X_enrichment_signal": "ID_anchors/pCyto_2X_enrichment.pixel_derived.signal_peaks.bed",
    "mCyto_2X_enrichment_signal": "ID_anchors/mCyto_2X_enrichment.pixel_derived.signal_peaks.bed",
    "mega_3X_enrichment": "ID_anchors/mega_3X_enrichment.fifth_mega3x.max_size.bed",
    "MEGA_2X_enrichment": "ID_anchors/MEGAp5_2X_enrichment.pixel_derived.signal_peaks.bed",
    "MEGAN93_2X_enrichment": "ID_anchors/MEGAN93p5_2X_enrichment.pixel_derived.signal_peaks.bed",
    "MEGAminus_2X_enrichment": "ID_anchors/MEGA_minus_ctrl_2X_enrichment.pixel_derived.signal_peaks.bed",
    "cyto_2x_enrichment": "ID_anchors/cyto_2x_enrichment.third_mCyto.max_size.bed",
}

id_anchors_dict = {}
for id_name, fname in id_anchor_fnames.items():
    id_anchors_dict[id_name] = pd.read_csv(fname, sep="\t")
    # ...
    print(f"loaded {len(id_anchors_dict[id_name]):5d} ID anchors {id_name:>20} in BED format ...")


In [ ]:
_Cyto_anchors = id_anchors_dict["pCyto_2X_enrichment_signal"]
_G1_anchors = id_anchors_dict["5hr_2X_enrichment_signal"]


cyto_not_in_g1 = bioframe.setdiff(  # Cyto specific one ...
    _Cyto_anchors.eval(
        """
        peak_start = peak_start - 100
        peak_end = peak_end + 100
        """
    ),
    _G1_anchors,
    cols1=["chrom","peak_start","peak_end"],
    cols2=["chrom","peak_start","peak_end"],
).reset_index(drop=True)


_Cyto_spec_anchors = bioframe.setdiff(  # Cyto specific one ...
    _Cyto_anchors,
    cyto_not_in_g1,
    cols1=["chrom","peak_start","peak_end"],
    cols2=["chrom","peak_start","peak_end"],
).reset_index(drop=True)

_G1_spec_anchors = bioframe.setdiff(  # G1 5hr specific one ...
    _G1_anchors,
    _Cyto_anchors,
    cols1=["chrom","peak_start","peak_end"],
    cols2=["chrom","peak_start","peak_end"],
).reset_index(drop=True)

len(_Cyto_spec_anchors) , len(_G1_spec_anchors), len(_Cyto_spec_anchors)+len(_G1_spec_anchors)
# we need bins/pixels for plotting - instead of genomic coordinates ...

# need a reference cooler for that to do genomic coords -> bins:
_clr = telo_clrs["m5hR1R2"]
# doing that using clr.offset and clr.extent functionality:
_G1_spec_anchors["bin_id"] = _G1_spec_anchors[['chrom', 'peak_start', 'peak_end']].apply(_clr.offset, axis=1, result_type="expand")
_G1_spec_anchors["bin_width"] = _G1_spec_anchors[['chrom', 'peak_start', 'peak_end']] \
    .apply(_clr.extent, axis=1, result_type="expand") \
    .apply(np.diff, axis=1, result_type="expand")[0]

# doing that using clr.offset and clr.extent functionality:
_Cyto_spec_anchors["bin_id"] = _Cyto_spec_anchors[['chrom', 'peak_start', 'peak_end']].apply(_clr.offset, axis=1, result_type="expand")
_Cyto_spec_anchors["bin_width"] = _Cyto_spec_anchors[['chrom', 'peak_start', 'peak_end']] \
    .apply(_clr.extent, axis=1, result_type="expand") \
    .apply(np.diff, axis=1, result_type="expand")[0]

In [ ]:

# in a specific region, and exposing importnat plotting parameters
def rectangles_at_anchors(anchors_df, region_extent, ax, halo=30_000, ext_width=0, **span_kwargs):
    # rectangle_kwargs = dict(lw=lw, ec=ec, fc=fc)
    # parse the tile
    _bin_start, _bin_end = region_extent
    # select only visible "boxes" :
    _the_anchors = anchors_df \
        .query("""(@_bin_start - @halo < bin_id < @_bin_end + @halo)""") \
        .eval("""
                b = bin_id - @_bin_start - @ext_width - 0.5
                b_width = bin_width + 2*@ext_width
            """)
    print(f"{len(_the_anchors)} anchors are visible out of {len(anchors_df)} ...")
    for b, w in _the_anchors[["b", "b_width"]].itertuples(index=False):
        ax.axvspan(b, b+w, **span_kwargs)


# Working on a Figure here

250kb (incl trans)
 - region1 = ('chr6', 105000000, 155000000)
 - region2 = ('chr7', 0, 56000000)

50kb
 - ('chr6',125200000, 145200000)

10kb assembly on- and off-diagonal
 - on diagonal #1: ('chr6', 129100000, 130500000)
 - on diagonal #2: ('chr6', 136100000, 138200000)

In [ ]:
mycmap = mpl.colors.LinearSegmentedColormap.from_list(
    name="myfall",
    colors=[ mpl.colors.to_hex(c) for c in plotting.PALETTES["fall"] ],
    N=2**16,
)
mycmap.set_bad('white',.5)
mycmap

In [ ]:

# def get_hm(
#     clr,
#     _idx_from,
#     _idx_to,
#     _jdx_from=None,
#     _jdx_to=None,
#     vmin=0.0002,
#     vmax=0.0125,
# ):
#     _jdx_from = _idx_from if _jdx_from is None else _jdx_from
#     _jdx_to = _idx_to if _jdx_to is None else _jdx_to
#     _mat = xr.DataArray(
#         clr.matrix()[_idx_from:_idx_to, _jdx_from:_jdx_to]
#     )
#     return tf.shade(
#         _mat,
#         cmap = mycmap,
#         how = lambda x,y: np.power(x, 1/3.),
#         span = (vmin, vmax),
#     ).to_pil(origin="upper")



# switch back to s
def get_hm(
    clr,
    _idx_from,
    _idx_to,
    _jdx_from=None,
    _jdx_to=None,
    vmin=0.0002,
    vmax=0.0125,
):
    #
    _jdx_from = _idx_from if _jdx_from is None else _jdx_from
    _jdx_to = _idx_to if _jdx_to is None else _jdx_to
    _mat = clr.matrix()[_idx_from:_idx_to, _jdx_from:_jdx_to]
    # need to NaN-out 2 diagonals ...
    if _idx_from == _jdx_from:
        for i in [-1,0,1]:
            _mat = fill_diag(_mat, np.nan, i)
        # ...
    _imshow_kwargs = dict(
        norm = LogNorm(vmin=vmin, vmax=vmax),
        cmap = mycmap,
        interpolation="nearest",
    )
    return _mat, _imshow_kwargs



# define regions for plotting ...
region1_10 = ('chr14', 53_500_000, 55_500_000)
region2_10 = ('chr14', 67_800_000, 69_000_000)

# vlims_dict = {
#     "250": dict(vmin=0.000005, vmax=0.002),
#     "50": dict(vmin=0.0001, vmax=0.01),
#     "10_1": dict(vmin=0.0002, vmax=0.0125),
#     "10_2": dict(vmin=0.0001, vmax=0.0025),
# }

vlims_dict = {
    "250": dict(vmin=0.00001, vmax=0.003),
    "50": dict(vmin=0.00004, vmax=0.025),
    # "10_1": dict(vmin=0.00006, vmax=0.06),
    # "10_2": dict(vmin=0.00006, vmax=0.06),
    "25_1": dict(vmin=0.000175, vmax=0.02),
    "25_2": dict(vmin=0.000175, vmax=0.02),
    "10_1": dict(vmin=0.00005, vmax=0.04),
    "10_2": dict(vmin=0.00005, vmax=0.005),
}


In [ ]:
# import matplotlib.pyplot as plt
import bbi
import matplotlib.lines as lines
from matplotlib.lines import Line2D
from matplotlib.patches import ConnectionPatch, Rectangle
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.inset_locator import BboxConnector

In [ ]:
margin = 0.2
spacing = 0.45*margin
matw = 0.95
cbarh = 0.07
cbarw = 0.7*matw
idbar = 1.5*cbarh  # make sure it is < margin+cbarh
patch_kwargs = dict(color="black", fc="none", lw=0.5)


_mev_color = "tab:blue"
_pev_color = "tab:red"

_region12_factor = (region2_10[2] - region2_10[1])/(region1_10[2] - region1_10[1])

# colorbar should have same width as the smaller heatmap part ...
cbarw = min(cbarw, _region12_factor*matw)

# The first items are for padding and the second items are for the axes, sizes are in inch.
h = [
    Size.Fixed(margin), Size.Fixed(0.125*matw), Size.Fixed(0.2*margin), # chrom schematic
    Size.Fixed(matw), Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    Size.Fixed(spacing), Size.Fixed(matw), Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    Size.Fixed(spacing), Size.Fixed(matw), Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    Size.Fixed(spacing), Size.Fixed(matw), Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    Size.Fixed(margin)
]
# goes from bottom to the top ...
v = [
    Size.Fixed(margin), Size.Fixed(cbarh),
    Size.Fixed(0.2*margin), Size.Fixed(matw),
    Size.Fixed(0.2*margin), Size.Fixed(matw*0.2),
    Size.Fixed(0.2*margin), Size.Fixed(matw),
]


# set figsize based on the tiling provided ...
fig_width = sum(_h.fixed_size for _h in h)
fig_height = sum(_v.fixed_size for _v in v)
fig = plt.figure(
    figsize=(fig_width, fig_height),
    # facecolor='lightblue'
)
print(f"figure size {fig_width=} {fig_height=}")

divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
axs = {}

# timepointa 10 kb region1 and region2 ....
for _i, _tp in enumerate(["t1", "t2", "t3", "t4"]):
    _nx = 4*_i+1+2
    axs[f"{_tp}_hmap1_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=1+2))
    axs[f"{_tp}_track_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=3+2))
    axs[f"{_tp}_hmap2_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=5+2))

    _nx = 4*_i+3+2
    axs[f"{_tp}_hmap1_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=1+2))
    axs[f"{_tp}_track_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=3+2))
    axs[f"{_tp}_hmap2_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=_nx, ny=5+2))


# chrom schematics
axs["chrom1"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=1+2))
axs["chrom2"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=5+2))


#################################
# chrom schematics on both axes ...
_regions_highlight = [("q", 52, 57, "dimgray"), ("q", 66, 68, "dimgray")]
_14pl = arms_dict["chr14_p"]
_14ql = arms_dict["chr14_q"]
_ww = 0.1*(_14pl+_14ql)
for ax in [axs["chrom1"], axs["chrom2"]]:
    _14, _14hs = get_chrom_path_wpathes(
        (0, 0),  # position of the centromere ...
        parm=_14pl,  # p arm length
        qarm=_14ql,  # q arm length
        width=_ww,  # width
        color="lightsteelblue",
        patches=_regions_highlight,
    )
    ax.set_axis_off()
    ax.add_artist(_14)
    for _h in _14hs:
        ax.add_artist(_h)
    ax.set_xlim(-1.0*_ww, 1.0*_ww)
    ax.set_ylim(-(_14ql)-(0.11+0.1)*_ww, (_14pl)+(0.11+0.1)*_ww)

##################################################
for ax_chrom, ax in zip([axs["chrom1"], axs["chrom2"]], [axs["t1_hmap1_reg1_bs10"], axs["t1_hmap2_reg1_bs10"]]):
    con = ConnectionPatch(
        xyA=(0.5*_ww, -52),
        coordsA=ax_chrom.transData,
        xyB=(0.0, 1.0),
        coordsB=ax.transAxes,
        **patch_kwargs,
    )
    ax.add_artist(con)
    con = ConnectionPatch(
        xyA=(0.5*_ww, -57),
        coordsA=ax_chrom.transData,
        xyB=(0.0, 0.0),
        coordsB=ax.transAxes,
        **patch_kwargs,
    )
    ax.add_artist(con)
# ##################################################


# define "prime" Sizes just to make way for fancy colorbarts ...
h_prime = [
    Size.Fixed(margin), Size.Fixed(0.125*matw), Size.Fixed(0.2*margin),  # chrom schematics
    Size.Fixed(matw - cbarw), Size.Fixed(cbarw),
    Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor-cbarw), Size.Fixed(cbarw),
    Size.Fixed(margin), Size.Fixed(matw), Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    Size.Fixed(margin), Size.Fixed(matw), Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    Size.Fixed(margin), Size.Fixed(matw), Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
]

divider_prime = Divider(fig, (0, 0, 1, 1), h_prime, v, aspect=False)
# colorbar axis :
axs["cbar_bs10_1"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=2+2, ny=1))
axs["cbar_bs10_2"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=5+2, ny=1))


# prime prime - for ID track ...
v_prime_prime = [
    Size.Fixed(margin + cbarh - idbar),     Size.Fixed(idbar),
    Size.Fixed(0.2*margin), Size.Fixed(matw),
    Size.Fixed(0.2*margin), Size.Fixed(matw*0.2),
    Size.Fixed(0.2*margin), Size.Fixed(matw),
]
divider_prime_prime = Divider(fig, (0, 0, 1, 1), h, v_prime_prime, aspect=False)
# ID track axis :
axs["t3_id_1"] = fig.add_axes(divider_prime_prime.get_position(), axes_locator=divider_prime_prime.new_locator(nx=9+2, ny=1))
axs["t3_id_2"] = fig.add_axes(divider_prime_prime.get_position(), axes_locator=divider_prime_prime.new_locator(nx=11+2, ny=1))

axs["t4_id_1"] = fig.add_axes(divider_prime_prime.get_position(), axes_locator=divider_prime_prime.new_locator(nx=13+2, ny=1))
axs["t4_id_2"] = fig.add_axes(divider_prime_prime.get_position(), axes_locator=divider_prime_prime.new_locator(nx=15+2, ny=1))


########################################
for k, ax in axs.items():
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)


_msample = "mMito"
_psample = "pMito"
clr10 = telo_clrs[_msample]
# and region1 and 2 @ 10kb ...
_i1_10, _i2_10 = clr10.extent(region1_10)
_j1_10, _j2_10 = clr10.extent(region2_10)
_track_size1_10 = _i2_10 - _i1_10
_track_size2_10 = _j2_10 - _j1_10

for ax_tag, ax in axs.items():
    if "track" in ax_tag:
        ax.axhline(0, color="grey", lw=0.5)
        ax.set_ylim(-2, 2)

# iterate over timepoints ...
for _tpoint, _msample, _psample in [
    ("t1", "mMito", "pMito"),
    ("t2", "mTelo", "pTelo"),
    ("t3", "mCyto", "pCyto"),
    ("t4", "m5hR1R2", "p5hR1R2"),
]:
    mtrack1_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_msample}.{binsize//1_000}kb.bw",region1_10[0],region1_10[1],region1_10[2], bins=_i2_10-_i1_10 ))
    mtrack2_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_msample}.{binsize//1_000}kb.bw",region2_10[0],region2_10[1],region2_10[2], bins=_j2_10-_j1_10 ))
    ptrack1_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_psample}.{binsize//1_000}kb.bw",region1_10[0],region1_10[1],region1_10[2], bins=_i2_10-_i1_10 ))
    ptrack2_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_psample}.{binsize//1_000}kb.bw",region2_10[0],region2_10[1],region2_10[2], bins=_j2_10-_j1_10 ))

    _ax_hmap2_reg1 = axs[f"{_tpoint}_hmap2_reg1_bs10"]
    _ax_hmap2_reg2 = axs[f"{_tpoint}_hmap2_reg2_bs10"]

    _ax_track_reg1 = axs[f"{_tpoint}_track_reg1_bs10"]
    _ax_track_reg2 = axs[f"{_tpoint}_track_reg2_bs10"]

    _ax_hmap1_reg1 = axs[f"{_tpoint}_hmap1_reg1_bs10"]
    _ax_hmap1_reg2 = axs[f"{_tpoint}_hmap1_reg2_bs10"]

    clr10 = telo_clrs[_msample]
    # 10 ...
    hm, hmargs = get_hm(clr10, _i1_10, _i2_10, **vlims_dict["10_1"])
    _ax_hmap2_reg1.imshow(hm, **hmargs)
    print(f"1st 510kb heatmap is {_track_size1_10} worth of pixels")
    _ax_track_reg1.plot(mtrack1_10, lw=0.7, color=_mev_color)
    _ax_track_reg1.plot(ptrack1_10, lw=0.7, color=_pev_color)
    _ax_track_reg1.set_xlim(0, _track_size1_10)
    # and region2 @ 10kb ...
    hm, hmargs = get_hm(clr10, _i1_10, _i2_10, _j1_10, _j2_10, **vlims_dict["10_2"])
    _ax_hmap2_reg2.imshow(hm, **hmargs)
    print(f"2nd 10kb heatmap is {_track_size2_10} worth of pixels")
    _ax_track_reg2.plot(mtrack2_10, lw=0.7, color=_mev_color)
    _ax_track_reg2.plot(ptrack2_10, lw=0.7, color=_pev_color)
    _ax_track_reg2.set_xlim(0, _track_size2_10)
    #  Now the depletion ...
    clr10 = telo_clrs[_psample]
    # 10 ...
    hm, hmargs = get_hm(clr10, _i1_10, _i2_10, **vlims_dict["10_1"])
    _ax_hmap1_reg1.imshow(hm, **hmargs)
    # and region2 @ 10kb ...
    hm, hmargs = get_hm(clr10, _i1_10, _i2_10, _j1_10, _j2_10, **vlims_dict["10_2"])
    _ax_hmap1_reg2.imshow(hm, **hmargs)


# # cyto_kwargs = dict(color="lightsteelblue", lw=0.5, ec="k")
# # G1_kwargs = dict(color="darkslategray", lw=0.5, ec="k")
# cyto_kwargs = dict(color="lightsteelblue", alpha=0.7,ec=None)
# G1_kwargs = dict(color="darkslategray", alpha=0.7,ec=None)
cyto_kwargs = dict(color="black", alpha=1,ec=None)
G1_kwargs = dict(color="black", alpha=1,ec=None)

# iterate over timepoints ...
for _tpoint, _msample, _psample, _anchor_df, kwargs in [
    # ("t3", "mCyto", "pCyto", _Cyto_anchors, cyto_kwargs),
    # ("t4", "m5hR1R2", "p5hR1R2", _G1_anchors, G1_kwargs),
    ("t3", "mCyto", "pCyto", _Cyto_spec_anchors, cyto_kwargs),
    ("t4", "m5hR1R2", "p5hR1R2", _G1_spec_anchors, G1_kwargs),
]:

    _ax_reg1 = axs[f"{_tpoint}_id_1"]
    _ax_reg2 = axs[f"{_tpoint}_id_2"]

    _ax_reg1.set_xlim(0,_track_size1_10)
    _ax_reg2.set_xlim(0,_track_size2_10)

    rectangles_at_anchors(_anchor_df, clr10.extent(region1_10), _ax_reg1, ext_width=-0.5, **kwargs)
    rectangles_at_anchors(_anchor_df, clr10.extent(region2_10), _ax_reg2, ext_width=-0.5, **kwargs)



# colorbars ...
# 10_1
_cax = axs["cbar_bs10_1"]
_vmin, _vmax = vlims_dict["10_1"]["vmin"], vlims_dict["10_1"]["vmax"]

plt.colorbar(
    cm.ScalarMappable(norm=LogNorm(vmin=_vmin, vmax=_vmax), cmap=mycmap),
    cax=_cax,
    orientation="horizontal",
)

_cax.xaxis.set_visible(True)
_ticks_power = range(
    np.ceil(np.log10(_vmin)).astype(int),
    np.floor(np.log10(_vmax)).astype(int)+1,
)
_cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
_cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
_cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)
_cax.minorticks_off()

# 10_2
_cax = axs["cbar_bs10_2"]
_vmin, _vmax = vlims_dict["10_2"]["vmin"], vlims_dict["10_2"]["vmax"]

plt.colorbar(
    cm.ScalarMappable(norm=LogNorm(vmin=_vmin, vmax=_vmax), cmap=mycmap),
    cax=_cax,
    orientation="horizontal",
)

_cax.xaxis.set_visible(True)
_ticks_power = range(
    np.ceil(np.log10(_vmin)).astype(int),
    np.floor(np.log10(_vmax)).astype(int)+1,
)
_cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
_cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
_cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)
_cax.minorticks_off()


plt.savefig("fig3C.svg", dpi=400)


## Some irrelevant fun with `interpolation` and `interpolation_stage` paramteres in imshow ...

In [ ]:
# 10 ...
hm, hmargs = get_hm(clr10, _i1_10-30, _i2_10+30, **vlims_dict["10_1"])
# 10 ...
for _inter in [ "nearest", "antialiased", "gaussian", "sinc"]:
    hmargs["interpolation"] = _inter
    f,axs = plt.subplots(ncols=2, figsize=(12,6))
    f.suptitle(f"interpolations {_inter=}")
    for i, _stage in enumerate(["data","rgba"]):
        hmargs["interpolation_stage"] = _stage
        ax = axs[i]
        ax.set_title(_stage)
        ax.imshow(hm, **hmargs)
        ax.set_axis_off()